In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd




## Get Top Places to Visit

In [7]:
def getSoup(link):
    r = requests.get(link)
    r.encoding = 'UTF-8'
    
    return BeautifulSoup(r.text,'lxml')


### Links


GETTING LIST OF COUNTRIES NAMES

In [8]:
numbers=range(0,len(countries))
countries_names = {countries[i][1] for i in numbers}



TRIPADVISOR LINKS FOR EVERY COUNTRY

In [9]:
links_google= ["https://www.google.com/search?q=tripadvisor+15+best+things+to+do+in+" + x for x in countries_names]

#### Country Things to do top 15

just fucntion

In [11]:
soup = getSoup('https://www.google.com/search?q=tripadvisor+best+things+to+do+in+slovakia')
h3=soup.find('h3', {'class':"r"})
print(h3)

<h3 class="r"><a href="/url?q=https://www.tripadvisor.cz/Attractions-g274922-Activities-Slovakia.html&amp;sa=U&amp;ved=0ahUKEwjKgb_oodXhAhXHwMQHHWdSAhYQFggUMAA&amp;usg=AOvVaw1hHLEAddZocny1Voc7vMSv">10 nejlepš�ch aktivit a pam�tek - <b>Slovensko</b> - <b>TripAdvisor</b></a></h3>


In [14]:
soup = getSoup('https://www.google.com/search?q=tripadvisor+best+things+to+do+in+denmark')
h3=soup.findAll('h3', {'class':"r"} )[2]
a=h3.contents[0]
url=a.get('href')
url=url.partition("Attractions")[2].partition(".html")[0]
ta = 'http://tripadvisor.com/Attractions' 
adress= ta+url
print(adress)



http://tripadvisor.com/Attractions-g189512-Activities-Denmark


In [276]:
a=h3.contents[0]
url=a.get('href')
url=url.partition("Attractions")[2].partition(".html")[0] #tripadvisor.cz
ta = 'http://tripadvisor.com/Attractions' 
adress= ta+url
print(adress)

for loop

In [32]:
links_tripadvisor=[]
for i in links_google:
    soup = getSoup(i)
    h3=soup.find('h3', {'class':"r"})
    a=h3.contents[0]
    url=a.get('href')
    url=url.partition("Attractions")[2].partition(".html")[0]
    ta = 'http://tripadvisor.com/Attractions' 
    adress= ta+url
    links_tripadvisor.append(adress)
print(links_tripadvisor)

['http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293828-Activities-Rwanda', 'http://tripadvisor.com/Attractions-g294481-Activities-Tuvalu', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g294307-Activities-Ecuador', 'http://tripadvisor.com/Attractions-g188553-Activities-The_Netherlands', 'http://tripadvisor.com/Attractions-g1767153-Activities-St_Helena_Island', 'http://tripadvisor.com/Attractions-g293843-Activities-Lusaka_Lusaka_Province', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293753-Activities-Tunisia', 'http://tripadvisor.com/Attractions-g147400-Activities-U_S_Virgin_Islands', 'http://tripadvisor.com/Attractions', 'http://tripadvisor.com/Attractions-g293961-Activities-Sri_Lanka', 'http://tripadvisor.com/Attractions-g293810-Activities-Malawi', 'http://tripadvisor.com/Attractions-g293747-Activities-Tanzania', 'http://tripadvisor.com/Attractions-g294064-Activi

### Attraction Titles 


In [35]:
attraction_titles_all=[]
for i in links_tripadvisor:    
    soup = getSoup(i)
    divs = soup.findAll('div', {'class':'listing_title'}) 
    text_chunks = {div.contents[1] for div in divs}
    attraction_titles_all.append({title.contents[0] for title in text_chunks})
    

In [36]:
x = input('ahoj')

ahoj


In [37]:
print(attraction_titles_all)

[set(), {'1000 Hills Distillery', 'Mt. Kigali', 'Nyamata Church', 'Kimironko Market', 'Niyo Art Gallery', 'Ivuka Arts Centre', 'Kigali City Tower', 'Go Gisenyi Tours', 'Kandt House Natural History Museum', 'Amahoro Stadium', 'Inema Art Center', 'Karisoke Research Center - Dian Fossey Camp', 'Camp Kigali Memorial', 'Librarie Ikirezi', 'Lake Burera and Lake Ruhondo', 'Murambi Genocide Memorial Center', 'Kigali Genocide Memorial', 'Rwanda Art Museum', 'National Museum of Rwanda', 'Ethnographic Museum', "Iby'iwacu Cultural Village", 'Our Lady of Kibeho Shrine', 'Virunga National Park', 'Muslim Quarter', 'Volcanoes National Park', 'Lake Kivu', 'Ntarama Church', 'Akagera National Park', 'Nyungwe National Park', 'Congo Nile Trail'}, {'Tuvalu National Library', "Tuvalu Women's Handicraft Centre", 'Tuvalu Philatelic Bureau', 'Funafuti Marine Conservation Area'}, set(), {'Fundacion Iglesia de la Compania', 'La Capilla Del Hombre', 'Las Grietas', 'Administracion Turistica del Parque Nacional Gala

IBA KÓD

In [16]:
    
soup = getSoup("http://tripadvisor.com/Attractions-g293961-Activities-Sri_Lanka")
divs = soup.findAll('div', {'class':'listing_title'}) 
text_chunks = {div.contents[1] for div in divs}
attraction_titles = {title.contents[0] for title in text_chunks}
print(attraction_titles)

{'Euphoria Spice and Herbal', 'Gangaramaya (Vihara) Buddhist Temple', 'Bentota Beach', 'Jami Ul-Alfar Mosque', 'Hikkaduwa Beach', 'Mulgirigala Raja Maha Vihara', 'Main Fish Market', 'Temple of the Sacred Tooth Relic', 'Udawalawe National Park', 'Muthurajawela Marsh', 'Yala National Park', 'Sinharaja Forest Reserve', 'Old Town of Galle and its Fortifications', 'Dutch Canal', 'Horton Plains National Park', 'Pidurangala Rock', 'Negombo Lagoon', 'Ancient City of Sigiriya', 'Negombo Fish Market', 'Mount Lavinia Beach', 'Kelaniya Raja Maha Vihara', "Little Adam's Peak", 'Mirissa Beach', 'Colombo National Museum', 'Japanese Peace Pagoda', 'Angurukaramulla Temple', 'Galle Fort', 'Koneswaram Temple', 'Sea Turtle Farm', "St. Mary's Church"}


## CLASS

In [ ]:
class Country:
    '''
    '''
    
    def __init__(self,link):
        self.link = link
        r = requests.get(link)
        r.encoding='UTF-8'
        self.soup = BeautifulSoup(r.text,'lxml')
    
    
    

In [4]:
#https://www.sitekickr.com/snippets/python/list-countries
countries = [
    ('US', 'United States'),
    ('AF', 'Afghanistan'),
    ('AL', 'Albania'),
    ('DZ', 'Algeria'),
    ('AS', 'American Samoa'),
    ('AD', 'Andorra'),
    ('AO', 'Angola'),
    ('AI', 'Anguilla'),
    ('AQ', 'Antarctica'),
    ('AG', 'Antigua And Barbuda'),
    ('AR', 'Argentina'),
    ('AM', 'Armenia'),
    ('AW', 'Aruba'),
    ('AU', 'Australia'),
    ('AT', 'Austria'),
    ('AZ', 'Azerbaijan'),
    ('BS', 'Bahamas'),
    ('BH', 'Bahrain'),
    ('BD', 'Bangladesh'),
    ('BB', 'Barbados'),
    ('BY', 'Belarus'),
    ('BE', 'Belgium'),
    ('BZ', 'Belize'),
    ('BJ', 'Benin'),
    ('BM', 'Bermuda'),
    ('BT', 'Bhutan'),
    ('BO', 'Bolivia'),
    ('BA', 'Bosnia And Herzegowina'),
    ('BW', 'Botswana'),
    ('BV', 'Bouvet Island'),
    ('BR', 'Brazil'),
    ('BN', 'Brunei Darussalam'),
    ('BG', 'Bulgaria'),
    ('BF', 'Burkina Faso'),
    ('BI', 'Burundi'),
    ('KH', 'Cambodia'),
    ('CM', 'Cameroon'),
    ('CA', 'Canada'),
    ('CV', 'Cape Verde'),
    ('KY', 'Cayman Islands'),
    ('CF', 'Central African Rep'),
    ('TD', 'Chad'),
    ('CL', 'Chile'),
    ('CN', 'China'),
    ('CX', 'Christmas Island'),
    ('CC', 'Cocos Islands'),
    ('CO', 'Colombia'),
    ('KM', 'Comoros'),
    ('CG', 'Congo'),
    ('CK', 'Cook Islands'),
    ('CR', 'Costa Rica'),
    ('CI', 'Cote D`ivoire'),
    ('HR', 'Croatia'),
    ('CU', 'Cuba'),
    ('CY', 'Cyprus'),
    ('CZ', 'Czech Republic'),
    ('DK', 'Denmark'),
    ('DJ', 'Djibouti'),
    ('DM', 'Dominica'),
    ('DO', 'Dominican Republic'),
    ('TP', 'East Timor'),
    ('EC', 'Ecuador'),
    ('EG', 'Egypt'),
    ('SV', 'El Salvador'),
    ('GQ', 'Equatorial Guinea'),
    ('ER', 'Eritrea'),
    ('EE', 'Estonia'),
    ('ET', 'Ethiopia'),
    ('FK', 'Falkland Islands (Malvinas)'),
    ('FO', 'Faroe Islands'),
    ('FJ', 'Fiji'),
    ('FI', 'Finland'),
    ('FR', 'France'),
    ('GF', 'French Guiana'),
    ('PF', 'French Polynesia'),
    ('TF', 'French S. Territories'),
    ('GA', 'Gabon'),
    ('GM', 'Gambia'),
    ('GE', 'Georgia'),
    ('DE', 'Germany'),
    ('GH', 'Ghana'),
    ('GI', 'Gibraltar'),
    ('GR', 'Greece'),
    ('GL', 'Greenland'),
    ('GD', 'Grenada'),
    ('GP', 'Guadeloupe'),
    ('GU', 'Guam'),
    ('GT', 'Guatemala'),
    ('GN', 'Guinea'),
    ('GW', 'Guinea-bissau'),
    ('GY', 'Guyana'),
    ('HT', 'Haiti'),
    ('HN', 'Honduras'),
    ('HK', 'Hong Kong'),
    ('HU', 'Hungary'),
    ('IS', 'Iceland'),
    ('IN', 'India'),
    ('ID', 'Indonesia'),
    ('IR', 'Iran'),
    ('IQ', 'Iraq'),
    ('IE', 'Ireland'),
    ('IL', 'Israel'),
    ('IT', 'Italy'),
    ('JM', 'Jamaica'),
    ('JP', 'Japan'),
    ('JO', 'Jordan'),
    ('KZ', 'Kazakhstan'),
    ('KE', 'Kenya'),
    ('KI', 'Kiribati'),
    ('KP', 'Korea (North)'),
    ('KR', 'Korea (South)'),
    ('KW', 'Kuwait'),
    ('KG', 'Kyrgyzstan'),
    ('LA', 'Laos'),
    ('LV', 'Latvia'),
    ('LB', 'Lebanon'),
    ('LS', 'Lesotho'),
    ('LR', 'Liberia'),
    ('LY', 'Libya'),
    ('LI', 'Liechtenstein'),
    ('LT', 'Lithuania'),
    ('LU', 'Luxembourg'),
    ('MO', 'Macau'),
    ('MK', 'Macedonia'),
    ('MG', 'Madagascar'),
    ('MW', 'Malawi'),
    ('MY', 'Malaysia'),
    ('MV', 'Maldives'),
    ('ML', 'Mali'),
    ('MT', 'Malta'),
    ('MH', 'Marshall Islands'),
    ('MQ', 'Martinique'),
    ('MR', 'Mauritania'),
    ('MU', 'Mauritius'),
    ('YT', 'Mayotte'),
    ('MX', 'Mexico'),
    ('FM', 'Micronesia'),
    ('MD', 'Moldova'),
    ('MC', 'Monaco'),
    ('MN', 'Mongolia'),
    ('MS', 'Montserrat'),
    ('MA', 'Morocco'),
    ('MZ', 'Mozambique'),
    ('MM', 'Myanmar'),
    ('NA', 'Namibia'),
    ('NR', 'Nauru'),
    ('NP', 'Nepal'),
    ('NL', 'Netherlands'),
    ('AN', 'Netherlands Antilles'),
    ('NC', 'New Caledonia'),
    ('NZ', 'New Zealand'),
    ('NI', 'Nicaragua'),
    ('NE', 'Niger'),
    ('NG', 'Nigeria'),
    ('NU', 'Niue'),
    ('NF', 'Norfolk Island'),
    ('MP', 'Northern Mariana Islands'),
    ('NO', 'Norway'),
    ('OM', 'Oman'),
    ('PK', 'Pakistan'),
    ('PW', 'Palau'),
    ('PA', 'Panama'),
    ('PG', 'Papua New Guinea'),
    ('PY', 'Paraguay'),
    ('PE', 'Peru'),
    ('PH', 'Philippines'),
    ('PN', 'Pitcairn'),
    ('PL', 'Poland'),
    ('PT', 'Portugal'),
    ('PR', 'Puerto Rico'),
    ('QA', 'Qatar'),
    ('RE', 'Reunion'),
    ('RO', 'Romania'),
    ('RU', 'Russian Federation'),
    ('RW', 'Rwanda'),
    ('KN', 'Saint Kitts And Nevis'),
    ('LC', 'Saint Lucia'),
    ('VC', 'St Vincent/Grenadines'),
    ('WS', 'Samoa'),
    ('SM', 'San Marino'),
    ('ST', 'Sao Tome'),
    ('SA', 'Saudi Arabia'),
    ('SN', 'Senegal'),
    ('SC', 'Seychelles'),
    ('SL', 'Sierra Leone'),
    ('SG', 'Singapore'),
    ('SK', 'Slovakia'),
    ('SI', 'Slovenia'),
    ('SB', 'Solomon Islands'),
    ('SO', 'Somalia'),
    ('ZA', 'South Africa'),
    ('ES', 'Spain'),
    ('LK', 'Sri Lanka'),
    ('SH', 'St. Helena'),
    ('PM', 'St.Pierre'),
    ('SD', 'Sudan'),
    ('SR', 'Suriname'),
    ('SZ', 'Swaziland'),
    ('SE', 'Sweden'),
    ('CH', 'Switzerland'),
    ('SY', 'Syrian Arab Republic'),
    ('TW', 'Taiwan'),
    ('TJ', 'Tajikistan'),
    ('TZ', 'Tanzania'),
    ('TH', 'Thailand'),
    ('TG', 'Togo'),
    ('TK', 'Tokelau'),
    ('TO', 'Tonga'),
    ('TT', 'Trinidad And Tobago'),
    ('TN', 'Tunisia'),
    ('TR', 'Turkey'),
    ('TM', 'Turkmenistan'),
    ('TV', 'Tuvalu'),
    ('UG', 'Uganda'),
    ('UA', 'Ukraine'),
    ('AE', 'United Arab Emirates'),
    ('UK', 'United Kingdom'),
    ('UY', 'Uruguay'),
    ('UZ', 'Uzbekistan'),
    ('VU', 'Vanuatu'),
    ('VA', 'Vatican City State'),
    ('VE', 'Venezuela'),
    ('VN', 'Viet Nam'),
    ('VG', 'Virgin Islands (British)'),
    ('VI', 'Virgin Islands (U.S.)'),
    ('EH', 'Western Sahara'),
    ('YE', 'Yemen'),
    ('YU', 'Yugoslavia'),
    ('ZR', 'Zaire'),
    ('ZM', 'Zambia'),
    ('ZW', 'Zimbabwe')]